<h1>Start of PHASE 1 - Filling the missing values - No feature selection - No Smoothing </h1>

In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('C:\\Users\\amir\\Desktop\\data.xlsx')

In [3]:
droped_data = data.drop(['year of credit scoring', 'year of financial statistices', 
           'Legal office/tax declaration', 'Legal office/Accredited auditor',
           'Legal office/corporate auidit', 'quantity score', 'qulity score', 
           'total score', 'credit rate', 'Lowe credit limit', 'middle credit limit', 
           'upper credit limit', 'export type', 'Audit report Reliability', 
           'credit expert 1 opinion', 'credit expert 2 opinion', 'risk expert 1 opinion', 
           'quality coins', 'Current period sales', 'Current period assets', 
           'Current period shareholder Equity'], axis=1)

In [4]:
droped_data_copy = droped_data.copy()

In [5]:
cols_with_missing = [col for col in droped_data.columns if droped_data[col].isnull().any()]

In [6]:
for col in cols_with_missing[:-5]:
    droped_data_copy = droped_data_copy[droped_data_copy[col].notnull()]

In [7]:
droped_data_copy = droped_data_copy[droped_data_copy['Last three years average exports'].notnull()]

In [8]:
X = droped_data_copy.drop(['Last three years average imports', 'Two-Prior period shareholder Equity'], axis=1)
y1, y2 = droped_data_copy['Last three years average imports'], droped_data_copy['Two-Prior period shareholder Equity']

In [9]:
X_train1 = X[y1.notnull()]
X_test1  = X[y1.isnull()]
y_train1 = y1[y1.notnull()]
y_test1  = y1[y1.isnull()]

In [10]:
from sklearn.ensemble import RandomForestRegressor
model1 = RandomForestRegressor(n_estimators=100, random_state=0)
model1.fit(X_train1, y_train1)
y_test = model1.predict(X_test1)

In [11]:
y1 = y_test
y1

array([4.14920841e+07, 1.36064057e+07, 2.04712000e+03, 3.13834000e+03,
       2.13800000e+02, 3.52301000e+04, 3.19262800e+04, 8.08615500e+04,
       1.31457030e+05, 4.35900000e+02, 7.43180000e+02, 1.26052000e+03,
       4.99910100e+04, 1.37680000e+02, 9.10510000e+02, 4.27196000e+03,
       1.45207160e+05, 2.82890680e+05, 4.25080000e+02, 3.15154700e+04,
       7.49074500e+03, 2.87416000e+03, 5.37082400e+04, 1.18450730e+05,
       1.14568250e+04, 3.85738500e+04, 4.98601000e+03, 1.70750000e+02,
       5.49923000e+03, 2.80772100e+05, 1.76790000e+02, 9.61605950e+05,
       5.05800000e+02, 5.37195000e+03, 4.92653800e+04, 2.61797000e+03,
       1.16996000e+03, 2.17128640e+04, 1.65466800e+04, 1.43249700e+04,
       1.02323580e+05, 5.09200600e+05, 8.74360300e+04, 8.11593320e+05,
       1.42393871e+06, 3.22805400e+05, 3.37978920e+05, 3.36260000e+02,
       3.37287700e+04, 2.92705000e+03, 3.24867700e+04, 2.13040000e+02])

In [12]:
X_train2 = X[y2.notnull()]
X_test2  = X[y2.isnull()]
y_train2 = y2[y2.notnull()]
y_test2  = y2[y2.isnull()]

In [13]:
model2 = RandomForestRegressor(n_estimators=100, random_state=0)
model2.fit(X_train2, y_train2)
y_test2 = model2.predict(X_test2)

In [14]:
droped_data_copy['Last three years average imports'][droped_data_copy['Last three years average imports'].isnull()] = y1

In [15]:
droped_data_copy['Last three years average imports'].isnull().sum()

0

In [16]:
y2 = y_test2
y2

array([0., 0., 0., 0., 0., 0.])

In [17]:
droped_data_copy['Two-Prior period shareholder Equity'][droped_data_copy['Two-Prior period shareholder Equity'].isnull()] = y2

In [18]:
droped_data_copy['Two-Prior period shareholder Equity'].isnull().sum()

0

In [19]:
from sklearn.model_selection import train_test_split
X = droped_data_copy.drop([' (worthy/nonworthy) y'], axis=1)
y = droped_data_copy[' (worthy/nonworthy) y']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [20]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
predict = model.predict(X_test)

In [21]:
from sklearn.metrics import accuracy_score

In [22]:
accuracy_score(y_test, predict)

0.8925619834710744

In [23]:
from sklearn.metrics import roc_auc_score
pred = model.predict_proba(X_test)
pred = [p[1] for p in pred]
roc_auc_score(y_test, pred)

0.8467122830440588

<h3>End of PHASE 1 - Filling the missing values - No feature selection - No Smoothing </h3>

<p>-------------------------------------------------------------------------------------------------------</p>

<h1>Start of PHASE 2 - Handling the Imbalanced Data - <span style=color:red>No Spliting for now</span> </h1>

In [24]:
data = droped_data_copy

In [37]:
from imblearn.over_sampling import SVMSMOTE
from collections import Counter

In [33]:
X, y = data.drop([' (worthy/nonworthy) y'], axis=1), data[' (worthy/nonworthy) y']

In [34]:
counter = Counter(y)
counter

Counter({1: 838, 0: 127})

In [35]:
oversample = SVMSMOTE()
X, y = oversample.fit_resample(X, y)
counter = Counter(y)
counter

Counter({1: 838, 0: 838})

In [43]:
data = pd.concat([X, y], axis=1)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model = RandomForestClassifier()
model.fit(X_train, y_train)
predict = model.predict(X_test)
print(accuracy_score(y_test, predict))
pred = model.predict_proba(X_test)
pred = [p[1] for p in pred]
print(roc_auc_score(y_test, pred))

0.9260143198090692
0.9806092600210248


<h1>Start of PHASE 3 - Feature Selection - <span style=color:red>No Smoothing</span> </h1>